In [3]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from hankel import HankelTransform

## Chen and Oldenburg 2004 implementation of the 2 layer system

In [13]:
# Electrode is placed at the interface between rho1 and rho2
class halfspace_2_layer:
    def __init__(self, rho1, rho2, h_overburden, tx_current, src_depth):
        """
        Returns the field magnitude of a buried electrode at src_depth
        self.src_depth
        @return: B_field magnitude
        """
        # Define a hankel transform of the first order
        self.hankelTF = HankelTransform(
            nu= 1,     # The order of the bessel function
            N = 120,   # Number of steps in the integration
            h = 0.03   # Proxy for "size" of steps in integration
        )
        # We'll have to tweak these parameters to optimize for 
        # speed and accuracy in the indefinite integral used in the defn of the Hankel Transform
        
        self.rho1 = rho1
        self.rho2 = rho2
        self.h_overburden = h_overburden
        
        self.src_depth = src_depth
        self.tx_current = tx_current

    # BField in the upper layer
    def _D_1(lbda):
        # Equation 17 from Chen and Oldenburg (2004)
        hnkl_h1 = self.hankelTF(lambda x: self.h_overburden, lbda)
                                
        bot = (1-np.exp(-2*lbda*hnkl_h1)) + self.rho2 / self.rho1 * (1+np.exp(-2*lbda*hnkl_h1))
        return self.tx_current * np.exp(-lbda * hnkl_h1) / 2 / np.pi / lbda / bot
    
    def _U_1(lbda):
        return -self._D_1(lbda)
    
    def _D_2(lbda):
        # Equation 17 from Chen and Oldenburg (2004)
        hnkl_h1 = self.hankelTF(lambda x: self.h_overburden, lbda)
        top = (1 + np.exp(-2 * lbda * hnkl_h1))
        bot = (1 - np.exp(-2 * lbda * hnkl_h1)) + self.rho2 / self.rho1 * (1 + np.exp(-2*lbda*hnkl_h1))
        return self.tx_current * top * self.rho2 / self.rho1 / 2 / np.pi / lbda / bot
    
    def _U_2(lbda):
        return 0


In [15]:
# Initialize a half-space
earth = halfspace_2_layer(rho1=100,
                  rho2=10,
                  h_overburden=50,
                  tx_current=2,
                  src_depth=25)